In [1]:
from parsezeeklogs import ParseZeekLogs
from glob import glob

import os

import pandas as pd

log 檔轉成 csv 檔

In [2]:
os.makedirs("zeek_csv", exist_ok=True)

for log_file in glob("zeek/*.log"):
    out_csv = log_file.replace(".log", ".csv").replace("zeek", "zeek_csv")
    with open(out_csv,"w") as outfile:
        zeekLogs = ParseZeekLogs(log_file, output_format="csv")
        outfile.write(zeekLogs.get_fields() + "\n")
        for log_record in zeekLogs:
            if log_record is not None:
                outfile.write(log_record + "\n")

conn 和 flowmeter 合併

In [3]:
conn = pd.read_csv("zeek_csv/conn.csv")
flowmeter = pd.read_csv("zeek_csv/flowmeter.csv")

pd.merge(conn, flowmeter, on="uid").to_csv("zeek_csv/conn_flowmeter.csv", index=False)

排除 analyzer 和 dns 的流量

In [2]:
conn_flowmeter = pd.read_csv("zeek_csv/conn_flowmeter.csv", low_memory=False)
analyzer = pd.read_csv("zeek_csv/analyzer.csv")
dns = pd.read_csv("zeek_csv/dns.csv")
# weird = pd.read_csv("zeek_csv/weird.csv")

conn_flowmeter = conn_flowmeter[~conn_flowmeter.uid.isin(analyzer.uid)]
conn_flowmeter = conn_flowmeter[~conn_flowmeter.uid.isin(dns.uid)]
# conn_flowmeter = conn_flowmeter[~conn_flowmeter.uid.isin(weird.uid)]

conn_flowmeter.to_csv("zeek_csv/filtered_conn_flowmeter.csv", index=False)

濾掉 netflow 中被 zeek 歸類為 analyzer 和 dns 的 flow

In [42]:
netflow = pd.read_csv("netflow/200702111400.csv", low_memory=False)
analyzer = pd.read_csv("zeek_csv/analyzer.csv")
dns = pd.read_csv("zeek_csv/dns.csv")

analyzer_dns = pd.merge(analyzer[["id.orig_h","id.resp_h","id.orig_p","id.resp_p"]], dns[["id.orig_h","id.resp_h","id.orig_p","id.resp_p"]], how="outer")
analyzer_dns.columns = ["sa","da","sp","dp"]

netflow_filtered = pd.merge(netflow, analyzer_dns, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
netflow_filtered.to_csv("netflow/200702111400_filtered.csv", index=False)
    